# Required libraries
# Langchain - Framework for generative AI
# Ageno - Agentic Framework
# Chroma is a Vector Database
# GoogleGenerativeAIEmbeddings for embedding

Importing Required libraries

In [23]:
# Required libraries

import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyMuPDFLoader  # PDF Loader
from langchain_text_splitters import RecursiveCharacterTextSplitter # Text splitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings # Embedding model
from langchain_chroma import Chroma # Vector DB
from agno.agent import Agent # Agent
from agno.models.groq import Groq # LLM Model
from agno.knowledge.langchain import LangChainKnowledgeBase 


# Loading Data -> Chunking -> Embedding -> Storing in Vector DB

In [25]:
# Loading Data -> Chunking 
data=PyMuPDFLoader('big_data.pdf').load() # Loading
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1024,chunk_overlap=50) # text splitter for chunking
chunks=text_splitter.split_documents(data) # Chunking
chunks

[Document(metadata={'producer': 'Skia/PDF m134 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': 'big_data.pdf', 'file_path': 'big_data.pdf', 'total_pages': 36, 'format': 'PDF 1.4', 'title': 'big_data', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content='What are some common skin diseases?  Some common skin diseases include acne, \neczema, psoriasis, rosacea, and skin cancer. These conditions can vary in severity and \nmay require medical treatment. \n How is acne formed?  Acne is formed when the hair follicles become plugged with oil \nand dead skin cells. This can lead to whiteheads, blackheads, and pimples. \n What is eczema and what are its symptoms?  Eczema is a chronic skin condition \ncharacterized by red, itchy, and scaly patches. It can affect any part of the body, but it \ntypically appears on the hands, feet, behind the knees, and in the elbow creases. \n What causes psorias

#  Embedding -> Storing in Vector DB

In [26]:
embeddings=GoogleGenerativeAIEmbeddings(model='models/embedding-001',google_api_key=os.getenv('GEMINI_API_KEY')) # embedding model for embedding
chroma=Chroma.from_documents(embedding=embeddings,documents=chunks) # Embedding and storing
chroma

# Creating a Knowledge Base for Agent

In [28]:
retriver=chroma.as_retriever()
knowledge_base=LangChainKnowledgeBase(retriever=retriver)
knowledge_base

LangChainKnowledgeBase(reader=None, vector_db=None, num_documents=5, optimize_on=1000, chunking_strategy=<agno.document.chunking.fixed.FixedSizeChunking object at 0x0000021719905BD0>, loader=None, vectorstore=None, search_kwargs=None, retriever=VectorStoreRetriever(tags=['Chroma', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x00000217199196D0>, search_kwargs={}))

# Defining Agent

In [ ]:
agent = Agent(
    model=Groq(id="llama-3.3-70b-versatile",api_key=os.getenv('GROQ_API_KEY')),
    description='Answer the query based on knowlwdge base',
    knowledge=knowledge_base,
    search_knowledge=True,
    markdown=False,
    
)
agent

Agent(model=Groq(id='llama-3.3-70b-versatile', name='Groq', provider='Groq', response_format=None, structured_outputs=False, supports_structured_outputs=False, tool_choice=None, show_tool_calls=None, tool_call_limit=None, _tools=None, _functions=None, _function_call_stack=None, system_prompt=None, instructions=None, tool_message_role='tool', assistant_message_role='assistant', frequency_penalty=None, logit_bias=None, logprobs=None, max_tokens=None, presence_penalty=None, seed=None, stop=None, temperature=None, top_logprobs=None, top_p=None, user=None, extra_headers=None, extra_query=None, request_params=None, api_key='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', base_url=None, timeout=None, max_retries=None, default_headers=None, default_query=None, http_client=None, client_params=None, client=None, async_client=None), name=None, agent_id=None, introduction=None, user_id=None, session_id=None, session_name=None, session_state=None, context=None, add_context=False, resolve

# Running the Agent

In [21]:
query="How is scabies diagnosed?"
print(agent.run(query).content)

Scabies is typically diagnosed through a physical examination, during which a healthcare provider will look for signs of mites, such as burrows or rashes. In some cases, a skin scraping may be performed to confirm the diagnosis. A skin scraping is a procedure in which a sample of skin cells is examined under a microscope to identify the presence of scabies mites or their eggs.
